<a href="https://colab.research.google.com/github/WURXPP/Hello-world/blob/master/LCimage_area.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://githubtocolab.com/giswqs/geemap/blob/master/examples/notebooks/113_image_area.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [ ]:
!pip install geemap

In [ ]:
import ee
import geemap
import os

Add ESA Land Cover data.

In [ ]:
Map = geemap.Map()
dataset = ee.ImageCollection("ESA/WorldCover/v200").first()
#Map.addLayer(dataset, {'bands': ['Map']}, 'ESA Land Cover')
#Map.add_legend(builtin_legend='ESA_WorldCover')
#Map

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=Q3MvVXGQa1aRLoeq-q2aXnhAzilfP7qLhMBSL5sEBso&tc=ZGvslC8fqoz5Lan8lOh4IeU4hpmOJkImZb9gmLDK43w&cc=v0fse7SpVgocekXpVOWx4v2N64mzeCLnXeLJzKVuP0M

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk4YXzeaGwkGz4tKW6NRIzqWEZHC0Xkehm4Kx_bdp-KZ2kVMkWJEqJ0

Successfully saved authorization token.


Calculate the area of each land cover type.

In [ ]:
gaul = ee.FeatureCollection(
  'FAO/GAUL_SIMPLIFIED_500m/2015/level0')
Ethiopia = gaul.filter(ee.Filter.eq('ADM0_NAME', 'Ethiopia'))
Map.addLayer(Ethiopia, {},'Ethiopia Boundaries')
Map.centerObject(Ethiopia, 6);
Map


Map(center=[8.621659671307777, 39.644984151349284], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
EthiopiaESA = dataset.clip(Ethiopia)
Map.addLayer(EthiopiaESA, {'bands': ['Map']}, 'ESA Land Cover:Ethiopia')
Map.add_legend(builtin_legend='ESA_WorldCover', position='bottomleft')
Map

Map(bottom=8098.0, center=[8.621659671307777, 39.644984151349284], controls=(WidgetControl(options=['position'…

Area calculation square km, by pixel counting

In [ ]:

df = geemap.image_area_by_group(
    EthiopiaESA, scale=1000, denominator=1e6, decimal_places=4, verbose=True
)
df

Calculating area for group 10 ...
Calculating area for group 20 ...
Calculating area for group 30 ...
Calculating area for group 40 ...
Calculating area for group 50 ...
Calculating area for group 60 ...
Calculating area for group 80 ...
Calculating area for group 90 ...


,area,percentage
group,,
10,154434.5207,0.1368
20,435461.5662,0.3856
30,234327.7738,0.2075
40,205200.4087,0.1817
50,1957.7190,0.0017
60,81619.2912,0.0723
80,8317.5624,0.0074
90,7989.5173,0.0071


Save the results to a CSV.

In [ ]:
df.to_csv('esa_areaEthiopia.csv')

**Dynamic world**

In [59]:
startDate = '2020-01-01'
endDate = '2021-01-01'

dw = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate(startDate, endDate).filterBounds(Ethiopia)

#Create a mode composite.
classification = dw.select('label')
dwComposite = classification.reduce(ee.Reducer.mode())
dwVisParams = {'min': 0,'max': 8, 'palette':['#0064c8', '#006400', '#ffff4c', '#0096a0', '#f096ff', '#ffbb22','#fa0000', '#b4b4b4', '#f0f0f0']}

# Clip the composite and add it to a new map.
Map1 = geemap.Map()
dw.Ethiopia=dwComposite.clip(Ethiopia)
Map1.addLayer(dw.Ethiopia, dwVisParams, 'Classified Composite')
# Add legend to the map

Map1.centerObject(Ethiopia, 6);

#Legend Dictionary
ee_class_table = """

Value	Color	Description
0	0064c8	Water
1	006400	Trees
2	ffff4c	Grass
3	0096a0	Wetland
4	f096ff	Crop
5	ffbb22	Shrub
6	fa0000	Urban
7	b4b4b4	Bare
8	f0f0f0	Snow and ice

"""
legend_dict = geemap.legend_from_ee(ee_class_table)
Map1.add_legend(title="Dynamic World Land Cover", legend_dict=legend_dict)
Map1

Map(center=[8.621659671307777, 39.644984151349284], controls=(WidgetControl(options=['position', 'transparent_…

Calculate the area of each land cover type.

In [60]:
#Extract some classes.
DWSubset_areas = dw.Ethiopia.updateMask(dw.Ethiopia.eq(0).Or(dw.Ethiopia.eq(1)).Or(dw.Ethiopia.eq(4)).Or(dw.Ethiopia.eq(3)));

#print(added_areas);
Map2 = geemap.Map()
Map2.addLayer(DWSubset_areas, dwVisParams, ' DWfour classes');
Map2.centerObject(Ethiopia, 6);
Map2

Map(center=[8.621659671307777, 39.644984151349284], controls=(WidgetControl(options=['position', 'transparent_…

exporting dw classes as asset

In [61]:
out_dir = os.path.join(os.path.expanduser('~/content/'), 'Downloads')
geemap.ee_export_image_to_drive( 
    DWSubset_areas, description='DWclasses', folder=out_dir, scale=30
)

#region = ee.Geometry.BBox(Ethiopia)
polygonBounds = Ethiopia.geometry()
#geemap.ee_export_image(DWSubset_areas, filename="DWclasses.tif", scale=30, region=polygonBounds)
#print('polygon.bounds(...) =', polygonBounds);
assetId = 'dw_subset'
geemap.ee_export_image_to_asset(DWSubset_areas, description='dwclasses', assetId=assetId, region=polygonBounds, scale=30)


In [56]:
print(out_dir)
!ls

/root/content/Downloads
esa_areaEthiopia.csv  sample_data


In [62]:
out_dir = os.path.join(os.path.expanduser('~/content/'), 'Downloads')
geemap.ee_export_image_to_drive( 
    DWSubset_areas, description='DWclasses', folder=out_dir, scale=30
)

#region = ee.Geometry.BBox(Ethiopia)
polygonBounds = Ethiopia.geometry()
#geemap.ee_export_image(DWSubset_areas, filename="DWclasses.tif", scale=30, region=polygonBounds)
#print('polygon.bounds(...) =', polygonBounds);
assetId = 'projects/ee-xupp2018/assets/DW/dw_subset'
geemap.ee_export_image_to_asset(DWSubset_areas, description='dwclasses', assetId=assetId, region=polygonBounds, scale=30)


In [ ]:
Export.image.toAsset({
  image:DWSubset_areas,
  description:'dwc',
  assetId:'projects/ee-xupp2018/assets/DW/dw_subset',
  region:polygonBounds,
  scale:100
})

Import from asset,  this is to check if there is no memory issue

In [ ]:
#code to be added # The following code is added by Panpan for test.

In [49]:
#region = ee.Geometry.BBox(Ethiopia)
polygonBounds = Ethiopia.geometry()
#geemap.ee_export_image(DWSubset_areas, filename="DWclasses.tif", scale=30, region=polygonBounds)
#print('polygon.bounds(...) =', polygonBounds);
assetId = 'projects/ee-xupp2018/assets/DW/dw_subset'
#geemap.ee_export_image_to_asset(DWSubset_areas, description='DWclasses', assetId=assetId, region=polygonBounds, scale=30)
geemap.ee_export_image_to_asset(DWSubset_areas, assetId=assetId, region=polygonBounds, scale=30)

In [48]:
dwLoad = ee.Image('projects/ee-xupp2018/assets/DW/dw_subset')
print(dwLoad)

ee.Image({
  "functionInvocationValue": {
    "functionName": "Image.load",
    "arguments": {
      "id": {
        "constantValue": "projects/ee-xupp2018/assets/DW/dw_subset"
      }
    }
  }
})


The following is added by Panpan for test.

In [45]:
areaDW = geemap.image_area_by_group(
    dwLoad, scale=1000, denominator=1e6, decimal_places=4, verbose=True
)
areaDW

EEException: ignored

below code does not run due to memory limit

In [22]:
trees = DWSubset_areas.eq(3);
trees = trees.rename(['trees_area']);

statsTotal = trees.reduceRegion(**{
    'reducer': ee.Reducer.count(),
    'geometry': Ethiopia.geometry(),
    'scale': 10,
    'maxPixels': 1e11
    });
totalPixels = statsTotal.get('trees_area');

#Mask 0 pixel values and count remaining pixels.
treesMasked = trees.selfMask();

treesStats = treesMasked.reduceRegion(**{
    'reducer': ee.Reducer.count(),
    'geometry': Ethiopia.geometry(),
    'scale': 10,
    'maxPixels': 1e10
    });
treeAreaPixels = treesStats.get('trees_area');
treeAreaPixels

#pixelCountStats = trees.reduceRegion(**{
#    'reducer': ee.Reducer.frequencyHistogram().unweighted(),
#   'geometry': Ethiopia.geometry(),
#   'scale': 10,
#   'maxPixels': 1e11
#});
# The result is a Dictionary.  Print it.
#print(pixelCountStats.getInfo())


RecursionError: ignored

GEE map area calculation code, does not run due to memory issue

In [23]:
areaDW = geemap.image_area_by_group(
    DWSubset_areas, scale=1000, denominator=1e6, decimal_places=4, verbose=True
)
areaDW

EEException: ignored

Save the results to a CSV.

In [ ]:
df.to_csv('nlcd_area.csv')

**GLAD**

In [ ]:
landmask = ee.Image("projects/glad/landBuffer4").mask();
m20 = ee.Image('projects/glad/GLCLU2020/LCLUC_2020').updateMask(landmask);
m20.clipped=m20.clip(Ethiopia)
visParamMap = {"min":0,"max":255,"palette":["FEFECC","FAFAC3","F7F7BB","F4F4B3","F1F1AB","EDEDA2","EAEA9A","E7E792","E4E48A",
"E0E081","DDDD79","DADA71","D7D769","D3D360","D0D058","CDCD50","CACA48","C6C63F","C3C337","C0C02F","BDBD27","B9B91E","B6B616",
"B3B30E","B0B006","609C60","5C985C","589558","549254","508E50","4C8B4C","488848","448544","408140","3C7E3C","387B38","347834",
"317431","2D712D","296E29","256B25","216721","1D641D","196119","155E15","115A11","0D570D","095409","065106","643700","643a00",
"643d00","644000","644300","644600","644900","654c00","654f00","655200","655500","655800","655a00","655d00","656000","656300",
"666600","666900","666c00","666f00","667200","667500","667800","667b00","ff99ff","FC92FC","F98BF9","F685F6","F37EF3","F077F0",
"ED71ED","EA6AEA","E763E7","E45DE4","E156E1","DE4FDE","DB49DB","D842D8","D53BD5","D235D2","CF2ECF","CC27CC","C921C9","C61AC6",
"C313C3","C00DC0","BD06BD","bb00bb","000003","000004","000005","BFC0C0","B7BDC2","AFBBC4","A8B8C6","A0B6C9","99B3CB","91B1CD",
"89AFD0","82ACD2","7AAAD4","73A7D6","6BA5D9","64A3DB","5CA0DD","549EE0","4D9BE2","4599E4","3E96E6","3694E9","2E92EB","278FED",
"1F8DF0","188AF2","1088F4","0986F7","55A5A5","53A1A2","519E9F","4F9B9C","4D989A","4B9597","499294","478F91","458B8F","43888C",
"418589","3F8286","3D7F84","3B7C81","39797E","37767B","357279","336F76","316C73","2F6970","2D666E","2B636B","296068","285D66",
"bb93b0","B78FAC","B48CA9","B189A6","AE85A2","AA829F","A77F9C","A47B99","A17895","9E7592","9A718F","976E8C","946B88","916885",
"8D6482","8A617F","875E7B","845A78","815775","7D5472","7A506E","774D6B","744A68","714765","de7cbb","DA77B7","D772B3","D46EAF",
"D169AB","CE64A8","CB60A4","C85BA0","C4579C","C15298","BE4D95","BB4991","B8448D","B54089","B23B86","AF3682","AB327E","A82D7A",
"A52976","A22473","9F1F6F","9C1B6B","991667","961264","000000","000000","000000",
"1964EB","1555E4","1147DD","0E39D6","0A2ACF","071CC8","030EC1","0000BA",
"0000BA","040464","0000FF","3051cf","000000","000000","000000","000000",
"000000","000000","000000","000000","000000","000000","000000","000000",
"000000","000000","000000","000000","000000","000000","000000","000000",
"547FC4","4D77BA","466FB1","4067A7","395F9E","335895","335896","335897","ff2828","ffffff","d0ffff","ffe0d0","ff7d00","fac800","c86400",
"fff000","afcd96","afcd96","64dcdc","00ffff","00ffff","00ffff","111133","000000"]};
Map3=geemap.Map()
Map3.centerObject(Ethiopia, 6);  
Map3.addLayer(m20.clipped,visParamMap,'2020 land cover and land use')
Map3

Map(center=[8.621659671307777, 39.644984151349284], controls=(WidgetControl(options=['position', 'transparent_…

subset GLAD

In [ ]:
#four classes
m20clippedSubset = m20.clipped.where(m20.clipped.gt(199).And(m20.clipped.lt(208)), 1).where(m20.clipped.gt(26).And(m20.clipped.lt(49)), 2).where(m20.clipped.eq(244), 3).where(m20.clipped.gt(102).And(m20.clipped.lt(149)), 4)#flooded 11%> veg cover
#1 is water, 2 is tree, 5m or higher trees 3 is crop and 4 is flooded 11%> veg cover
masked = m20clippedSubset.updateMask(m20clippedSubset.lte(4).And(m20clippedSubset.gt(0))) # remove 0s
    
#water =between (200 and 207))
#trees =  between 27 and48   #5m or higher trees
#crop = (244)
#wetland= between 103 148 #flooded 11%> veg cover
gladVisParams = {'min': 1,'max': 4, 'palette':['#0064c8', '#006400', '#f096ff', '#0096a0']}
#Legend Dictionary
ee_class_table1 = """

Value	Color	Description
1	0064c8	Water
2	006400	Trees
3	f096ff	Crop
4	0096a0	Wetland

"""
legend_dict1 = geemap.legend_from_ee(ee_class_table1)
#plot it
Map4=geemap.Map()
Map4.centerObject(Ethiopia, 6); 
Map4.addLayer(masked, gladVisParams,'2020 glad clip')
Map4.add_legend(title="2020 glad", legend_dict=legend_dict1)
Map4

Map(center=[8.621659671307777, 39.644984151349284], controls=(WidgetControl(options=['position', 'transparent_…

GLAD area calculation

In [ ]:
dfGLAD = geemap.image_area_by_group(
    masked, scale=1000, denominator=1e6, decimal_places=4, verbose=True
)
dfGLAD
#area percentage does not include the NA area

Calculating area for group 1 ...
Calculating area for group 2 ...
Calculating area for group 3 ...
Calculating area for group 4 ...


,area,percentage
group,,
1,17497.8081,0.0452
2,189808.7627,0.4905
3,154721.7124,0.3998
4,24938.1494,0.0644
